# Work to be done before collecting data:

In [1]:
import os
from tqdm.auto import tqdm
import pandas as pd


## 1.  Aller au Service_Suite et collecter RMA(& autres infos)
dans le fichier SS_ref.xlsx

In [2]:
data_ss = pd.read_excel('../Ref_2351573/SS_New.xlsx')
data_ss.loc[data_ss['elf_days']==0, 'adjusted_earlylife_failure']='FOA'
data_ss.loc[(data_ss['elf_days']>0) & (data_ss['elf_days']<180), 'adjusted_earlylife_failure']='ELF'
data_ss.loc[(data_ss['elf_days']>=180), 'adjusted_earlylife_failure']='IPFR180'

In [3]:
data_ss.to_excel('SS_4_classes.xlsx')

In [4]:
rma_FOA = data_ss.loc[data_ss['adjusted_earlylife_failure']=='FOA', 'rma_no']
rma_FOA.to_excel('RMA_FOA.xlsx')

In [5]:
rma_NF = data_ss.loc[data_ss['adjusted_earlylife_failure']!='FOA', 'rma_no']
rma_NF.to_excel('RMA_NF.xlsx')

**Attention, il faut supprimer la pemière colonne et la première ligne ici!**


Changer Item number s'il a besoins

In [16]:
import Rapide_search_v2 as search
# ~ 22 mins
table = '''SELECT [DW_TXN_KEY] [indices],[Transaction Day],[Transaction Type],[Transaction Description] ,
[Transaction Sub-Type] ,[Transaction Category] ,[Txn Quantity],[Inventory Org],[Subinventory Code],[ITEM],
[Item Description],[Job Number],[SO Number],[PO Number],[LPN],[FE Txn Group] [RMA],[SO PO Link],[Job Open Date],
[PO Type],[FE SSO], [FE ORG],[System ID],[System Install Date],[Customer Number],[Customer Name],[Customer Country Code],
[Customer Pole],[POLE],[LOCATOR],[Transfer Locator],[Vendor Name],[Vendor Site Code], [SO Type], [Reason Name],
[PO Header Description],[Failure Description],[Waybill], [Freight Carrier], [SO PO Link Type],[Obso Pivot Code]
FROM [dbo].[vwAllTxns_v3] 
Where (1=1) and ([ITEM] = '2351573-R' or [ITEM] = '2351573') '''
table2 = '''SELECT [DW_ERT_KEY] [indices],[ITEM], [Item Description], [Current Item Type], [Current First Put-Away Date (FPAD)],
[Current Item Status], [Vendor GSL Number], [Vendor Name],[Vendor Site Code], [PO1 PO Number], [PO1 Need By Date], 
[PO1 PO Header Description], [PO1 Approved Date], [PO1 Creation Date],  [PO1 Shipment Last Update Date], [PO1 Receipt Date], 
[PO1 PO Type], [PO1 PO Type DFF], [PO1 Authorization Status], [PO2 PO Number], [SO1 Shipping Warehouse], [FE Number], [FE SSO], 
[Failure Code],[Part Failure Analysis],[FE Comments],[Supplier Comments],[Analyst Comments], [Repair Status], [Repair Action], 
[Job Number], [SO1 SO Number], [SO1 SO Status], [SO1 Waybill], [SO1 Order Date], [SO1 Ship Date], [SO2 SO Number], 
[Serial Number], [Part Arrival Date], [Part Repaired Date]
FROM [dbo].[vwErt] 
Where (1=1) and ([ITEM] = '2351573-R' or [ITEM] = '2351573') '''
TXN = search.run_query_sqldb(f'{table}',search.open_conn_FBI())
ERT = search.run_query_sqldb(f'{table2}',search.open_conn_FBI())
# sdata = ERT[ERT['Serial Number']=='10990497']
# print(sdata)
search.menu()


------------------------------------------------------------------
------------------------Multiple tracing mode ----------------------------
------------------------------------------------------------------
---------------------Please Upload Your File------------------------
-------------------------File Uploaded :)----------------------------
--------------------Please Select directory folder-------------------
------------------------Multiple tracing mode repair: nan ----------------------------
Search in progress...
Connecting to database...
connected
ERROR for parameter repair : nan
--- 0.0 seconds ---
------------------------Multiple tracing mode repair: nan ----------------------------
Search in progress...
Connecting to database...
connected
ERROR for parameter repair : nan
--- 0.0 seconds ---
------------------------Multiple tracing mode repair: nan ----------------------------
Search in progress...
Connecting to database...
connected
ERROR for parameter repair : nan
--- 0.0 

## 2.Collection des POs pour les pièces Debrief(not FOA)

In [2]:
def PO_collection(file_dir):
    all_files_list = os.listdir(file_dir)
    tracing_files = [i for i in all_files_list if os.path.isfile(os.path.join(file_dir,i)) and i.startswith('tracing')]
    liste_po = []
    liste_rma = []
    for file in tqdm(tracing_files):
        single_tracing = pd.read_excel(os.path.join(file_dir, file), sheet_name='txn history')
        single_tracing_rma = pd.read_excel(os.path.join(file_dir, file), sheet_name='Summary', index_col=0)

        df_PO = single_tracing[['Transaction Sub-Type', 'PO Number', 'Job Number']]
        po = df_PO[df_PO['Transaction Sub-Type'].str.contains('RDE Repair')]['PO Number']
        if len(po)==0:
            po = df_PO[df_PO['Transaction Sub-Type'].str.contains('GPO Repair')]['Job Number']
        try:
            po_val = po.values[0]
        except:
            # print(file)
            po_val = None
        liste_po.append(po_val)

        rma = single_tracing_rma.loc['Key_value', 'Values']
        liste_rma.append(rma)
        # print(file, po_val, rma)
    return liste_po, liste_rma


In [3]:
os.chdir('c:/Users/223102584/Box/Doc_Xin/Sujet_Principale/Coil')
list_po, liste_rma = PO_collection('./Tracing_RMA_NF/')

  0%|          | 0/1800 [00:00<?, ?it/s]

In [5]:
single_tracing = pd.read_excel('./Tracing_RMA_NF/tracing rma_123132120.xlsx', sheet_name='txn history')
df_PO = single_tracing[['Transaction Sub-Type', 'PO Number', 'Job Number']]
po = df_PO[df_PO['Transaction Sub-Type'].str.contains('RDE Repair')]['PO Number']
if len(po)==0:
    po = df_PO[df_PO['Transaction Sub-Type'].str.contains('GPO Repair')]['Job Number']
try:
    po_val = po.values[0]
except:
    # print(file)
    po_val = None

In [6]:
po_val

301667325.0

In [63]:
count=0
for i in range(len(list_po)):
    if list_po[i]==None:
        count = count+1

In [64]:
count

2585

## 3. RMA <-> PO

In [66]:
import Data_collection_ALL as datacol

In [5]:
rma_po = pd.DataFrame()
rma_po['Forward_rma'] = liste_rma
rma_po['PO'] = list_po

In [ ]:
rma_po.to_excel('rma_po.xlsx')

In [7]:
SS = pd.read_excel('SS_4_classes.xlsx')
rma_po['Forward_rma'] = rma_po['Forward_rma'].astype(str)
SS['rma_no']= SS['rma_no'].astype(str)

In [14]:
emm = rma_po.merge(SS, how='left', left_on='Forward_rma', right_on='rma_no')
emm.to_excel('compaire.xlsx')

In [16]:
coldh = pd.read_excel('itemmtype.xlsx')
coldh['rma_no'] = coldh['rma_no'].astype(str)
emm['Forward_rma'] = emm['Forward_rma'].astype(str)
emm2 = emm.merge(coldh, how='left', left_on='Forward_rma', right_on='rma_no')
emm2.to_excel('temp.xlsx')

In [69]:
while list_po.count(None):
    list_po.remove(None)
df_po = pd.DataFrame(list_po)

In [70]:
df_po.to_excel('PO.xlsx')

## Création Summary

In [2]:
file_dir = './Tracing_FOA/'
# print(os.getcwd())
all_files_list = os.listdir(file_dir)
T_sum = pd.DataFrame()
i = 0
tracing_files = [i for i in all_files_list if os.path.isfile(os.path.join(file_dir,i)) and i.startswith('tracing')]
for file in tracing_files:
    sum_single = pd.read_excel(os.path.join(file_dir, file), sheet_name='Summary')
    sum_single = sum_single.T
    sum_single.columns =sum_single.iloc[0]
    sum_single = sum_single.drop(sum_single.index[0])
    T_sum = pd.concat([T_sum, sum_single])
T_sum

Columns,Input_Key,Key_value,Warning,Serial Number,Part Status,Customer Country,Customer POLE,Process_owner,Number of Warehouse,Number of returns,...,FE comments (LAST REPAIR),Supplier Comments (LAST REPAIR),Analyst Comments (LAST REPAIR),Last Localisation,Future Key search,Future value search,Since,Locator,Subinventory,POLE
Values,RMA,327074473,"Based on the retrieved data from FBI, there is...",35118552C,Debrief,FR,EU,R00>SHU>R00,1,1,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN
Values,RMA,333504848,"Based on the retrieved data from FBI, there is...",35A14F50C,Debrief,FR,EU,U00>R00>SRO>R00,2,1,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN
Values,RMA,333514771,This tool uses data from the GPRS datawarehous...,NaN,Not Consumed,AR,AM,U00>U43>SAR,2,0,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN
Values,RMA,335719384,This tool uses data from the GPRS datawarehous...,A961100058,FOA,US,AM,R00>U00>SUS>U00,2,1,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN
Values,RMA,336012730,This tool uses data from the GPRS datawarehous...,35A14085B,Debrief,RU,EU,U00>R00>R23>SRU>R23>SRU,3,1,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Values,RMA,353071440,"Based on the retrieved data from FBI, there is...",NaN,FOA,GB,EU,R00>SGB>R10,2,2,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN
Values,RMA,355488089,"Based on the retrieved data from FBI, there is...",NaN,FOA,US,AM,U08>SUS>U00,2,1,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN
Values,RMA,356218589,"Based on the retrieved data from FBI, there is...",NaN,FOA,CN,AS,A18>SCN>A18,1,1,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN
Values,RMA,356284043,"Based on the retrieved data from FBI, there is...",NaN,FOA,TR,EU,R30>STR>R30,1,2,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NaN


In [3]:
T_sum = T_sum.reset_index(drop=True)

In [4]:
T_sum.to_excel('./Tracing_FOA/rma_B_all_parts.xlsx')